In [17]:
import requests
import math
import os
import csv
import numpy as np
import pandas as pd

## Functions

In [2]:
def getRow(trip):
    '''
    Flattens a BlaBlaCar trip object into the plain list we want to upload to CARTO
    '''
    return {
            'url'     : trip['links']['_front'], 
            'from_lat': trip['departure_place']['latitude'], 
            'from_lon': trip['departure_place']['longitude'], 
            'to_lat'  : trip['arrival_place']['latitude'],
            'to_lon'  : trip['arrival_place']['longitude'],
            'car_comfort' : trip['car']['comfort_nb_star'] if 'car' in trip else '' ,
            'car_maker'   : trip['car']['make'] if 'car' in trip else '',
            'dep_date'    : trip['departure_date'],
            'distance'    : trip['distance']['value'],
            'duration'    : trip['duration']['value'],
            'price'       : trip['price_with_commission']['value'],
            'seats'       : trip['seats']
            }

In [10]:
def getTrips(city):  #Berlin-Dresden
    '''
    Iterates over the BlaBlaCar /trips/ endpoint to retrieve all the pages
    '''
    trips = []
    page = 1
    
    URL = "https://public-api.blablacar.com/api/v3/trips?from_coordinate=52.520008,13.404954&to_coordinate=51.050407,13.737262&locale=de-DE&currency=EUR&key=xcP0TPgR57BBbCUxjCI8G9S36kRVVnaO"
    KEY = os.getenv('BLABLACAR_API')
    CUR = "EUR"
    HEADERS = {
        'Content-Type': "application/json",
        'Cache-Control': "no-cache"
    }
    QS_BASE = {"key": KEY,"cur": CUR, "limit": 100} 

    querystring = dict( { 'page' : page, "fn": city, }, **QS_BASE)
    response = requests.request("GET", URL, headers=HEADERS, params=querystring)
    rj = response.json()

    while 'trips' in rj:
        #print('  Got page {}'.format(page))
        trips.extend(rj['trips'])
        page = page + 1

        querystring = dict( { 'page' : page, "fn": city }, **QS_BASE)
        response = requests.request("GET", URL, headers=HEADERS, params=querystring)
        rj = response.json()
        
    return trips







## Process cities

Process a list of cities to get all the trips and generates as a result a CSV ready to be used at CARTO

In [11]:
cities = ['Berlin']

In [12]:
all_trips = []

for city in cities:
    print('Getting the trips from BlaBlaCar API from {}...'.format(city))
    trips = getTrips(city)        
    print('{} trips retreived'.format(len(trips)))        
    all_trips.append({ 'city':city,'trips': trips})
    print('')

Getting the trips from BlaBlaCar API from Berlin...
1610 trips retreived



In [14]:
print(all_trips)

[{'city': 'Berlin', 'trips': [{'link': 'https://www.blablacar.de/trip?source=CARPOOLING&id=2193127950-potsdam-drezda', 'waypoints': [{'date_time': '2021-05-12T19:30:00', 'place': {'city': 'Potsdam', 'address': 'Kaufland Potsdam-Waldstadt, Potsdam', 'latitude': 52.364434, 'longitude': 13.093832, 'country_code': 'DE'}}, {'date_time': '2021-05-12T21:30:00', 'place': {'city': 'Dresden', 'address': 'Elbepark Dresden, Dresden', 'latitude': 51.084884, 'longitude': 13.694924, 'country_code': 'DE'}}], 'price': {'amount': '10.00', 'currency': 'EUR'}, 'vehicle': {'make': 'HONDA', 'model': 'CIVIC'}, 'distance_in_meters': 199722, 'duration_in_seconds': 7200}, {'link': 'https://www.blablacar.de/trip?source=CARPOOLING&id=2192422725-schoenefeld-dresden', 'waypoints': [{'date_time': '2021-05-12T22:20:00', 'place': {'city': 'Schönefeld', 'address': 'Flughafen Berlin Brandenburg Willy Brandt, Schönefeld', 'latitude': 52.364441, 'longitude': 13.509853, 'country_code': 'DE'}}, {'date_time': '2021-05-13T00:

Once get all the trips, save them into a single CSV file

In [13]:
filename = 'blablatripsDE2.csv'
print('Generating {}...'.format(filename))
with open(filename, 'w', newline='') as csvfile:
    fieldnames = [
        'city', 'url', 'from_lat', 'from_lon', 'to_lat', 'to_lon', 'car_comfort', 
        'car_maker', 'dep_date', 'distance', 'duration', 'price', 'seats'
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for group in all_trips:
        city = group['city']
        trips = group['trips']
        try:
            for trip in trips:
                row = getRow(trip)
                writer.writerow(dict({'city':city},**row))
        except KeyError as e:
            print(e)
            print(trip)

print('Done!')

Generating blablatripsDE2.csv...
'links'
{'link': 'https://www.blablacar.de/trip?source=CARPOOLING&id=2193127950-potsdam-drezda', 'waypoints': [{'date_time': '2021-05-12T19:30:00', 'place': {'city': 'Potsdam', 'address': 'Kaufland Potsdam-Waldstadt, Potsdam', 'latitude': 52.364434, 'longitude': 13.093832, 'country_code': 'DE'}}, {'date_time': '2021-05-12T21:30:00', 'place': {'city': 'Dresden', 'address': 'Elbepark Dresden, Dresden', 'latitude': 51.084884, 'longitude': 13.694924, 'country_code': 'DE'}}], 'price': {'amount': '10.00', 'currency': 'EUR'}, 'vehicle': {'make': 'HONDA', 'model': 'CIVIC'}, 'distance_in_meters': 199722, 'duration_in_seconds': 7200}
Done!
